In [1]:
from __future__ import absolute_import, division, print_function

# Import TensorFlow >= 1.10 and enable eager execution
import tensorflow as tf
tf.enable_eager_execution()

import librosa
import os
import time
import numpy as np
import glob
import matplotlib.pyplot as plt
import librosa
#import simpleaudio as sa
import math
from IPython import display
import sys

print("Finished imports")

C:\Users\1002983\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Finished imports


In [10]:
# Converting into spectrograms
# Spectrograms are stored into spectrograms.tfrecord
nsynth = tf.data.TFRecordDataset("nsynth-train.tfrecord")

read_features = {
    'note': tf.FixedLenFeature([], dtype=tf.int64),
    'note_str': tf.FixedLenFeature([], dtype=tf.string),
    'instrument': tf.FixedLenFeature([], dtype=tf.int64),
    'instrument_str': tf.FixedLenFeature([], dtype=tf.string),
    'pitch': tf.FixedLenFeature([], dtype=tf.int64),
    'velocity': tf.FixedLenFeature([], dtype=tf.int64),
    'sample_rate': tf.FixedLenFeature([], dtype=tf.int64),
    'audio': tf.VarLenFeature(dtype=float),
    'instrument_family': tf.FixedLenFeature([], dtype=tf.int64),
    'instrument_family_str': tf.FixedLenFeature([], dtype=tf.string),
    'instrument_source': tf.FixedLenFeature([], dtype=tf.int64),
    'instrument_source_str': tf.FixedLenFeature([], dtype=tf.string)
}

with tf.python_io.TFRecordWriter('spectrograms.tfrecord') as writer:
    acoustic = 0
    total = 0
    for b in nsynth:
        data = tf.parse_single_example(serialized=b, features=read_features)
        
        total += 1
        if(total%1000==0): 
            print("Acoustic: {}/{}".format(acoustic, total), end='\r')
            
        if(data['instrument_source'].numpy()==0):
            acoustic += 1
            audio = data['audio'].values.numpy()*15000
            audio = np.array(audio)
            spectrogram = librosa.stft(audio, 1022)            
            magnitudes = np.log(np.abs(spectrogram) + 1.2664166e-14).reshape(1, 512, 251)
            phases = np.angle(spectrogram).reshape(1, 512, 251)
            magnitudes = 2*(magnitudes+32)/(16+32)-1
            phases = phases / (math.pi)
            spec = np.pad(np.concatenate([magnitudes, phases], 2), ((0, 0), (0, 0), (0,5)), mode='constant', constant_values=0).flatten()

            example = tf.train.Example(features=tf.train.Features(feature={
                'note': tf.train.Feature(int64_list=tf.train.Int64List(value=[data['note'].numpy()])),
                'note_str': tf.train.Feature(bytes_list=tf.train.BytesList(value=[data['note_str'].numpy()])),
                'instrument': tf.train.Feature(int64_list=tf.train.Int64List(value=[data['instrument'].numpy()])),
                'instrument_str': tf.train.Feature(bytes_list=tf.train.BytesList(value=[data['instrument_str'].numpy()])),
                'pitch': tf.train.Feature(int64_list=tf.train.Int64List(value=[data['pitch'].numpy()])),
                'velocity': tf.train.Feature(int64_list=tf.train.Int64List(value=[data['velocity'].numpy()])),
                'sample_rate': tf.train.Feature(int64_list=tf.train.Int64List(value=[data['sample_rate'].numpy()])),
                'spectrogram': tf.train.Feature(float_list=tf.train.FloatList(value=spec)),
                'instrument_family': tf.train.Feature(int64_list=tf.train.Int64List(value=[data['instrument_family'].numpy()])),
                'instrument_family_str': tf.train.Feature(bytes_list=tf.train.BytesList(value=[data['instrument_family_str'].numpy()])),
                'instrument_source': tf.train.Feature(int64_list=tf.train.Int64List(value=[data['instrument_source'].numpy()])),
                'instrument_source_str': tf.train.Feature(bytes_list=tf.train.BytesList(value=[data['instrument_source_str'].numpy()]))
            }))
            
            writer.write(example.SerializeToString())
            
print("\nDone")

(512, 251)


IndexError: index 512 is out of bounds for axis 1 with size 512